In [34]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [35]:
## Load the trained model, scalar pickle and onehot
model=load_model('churn_model.h5')

## load the encoder and scalar pickle
with open('oneHotEncoder_geo.pkl','rb') as f:
    label_encoder_geo=pickle.load(f)
with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)  
with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)

In [36]:
input_data = {
  'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 50000
}

In [37]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


d:\misc\code\GenAI\ANNClass\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [38]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [39]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [40]:
input_df = pd.concat([input_df,geo_encoded_df],axis=1)
input_df.drop(['Geography'],axis=1,inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [41]:
## scaling data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [42]:
## prediction
my_prediction = model.predict(input_scaled)
my_prediction

1/1 [==============================] - 0s 223ms/step


array([[0.02525376]], dtype=float32)

In [43]:
prediction_prob = my_prediction[0][0]
prediction_prob

0.025253762

In [44]:
if prediction_prob > 0.5:
    print("Customer is likely to churn.")
else:
    print("Customer is not likely to churn.")

Customer is not likely to churn.
